<a href="https://colab.research.google.com/github/maxi9113/colab_notebook/blob/main/Class1_Lucas_Kanade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lucas–Kanade Optical Flow
https://en.wikipedia.org/wiki/Lucas%E2%80%93Kanade_method

Este cuaderno permite:
- Subir un video desde tu computadora,
- Seleccionar una **región de interés (ROI)** `pts = ((0,0),(100,100))`,
- Escoger dos frames (t y t+Δ) con un deslizador,
- Calcular el **flujo óptico (u,v)** con:
  - (1) una **implementación manual** del método Lucas–Kanade, y
  - (2) la versión **OpenCV (Farneback)**,
- Visualizar las flechas de movimiento de ambos algoritmos.

---
### Ecuación de Lucas–Kanade

El método de Lucas–Kanade busca las velocidades \( u, v \) que minimizan el error entre dos imágenes consecutivas:

$$
\min_{u,v} \sum_{x,y \in W} [I_x(x,y)\,u + I_y(x,y)\,v + I_t(x,y)]^2
$$

Derivando y despejando, obtenemos el sistema lineal:

$$
\begin{bmatrix}
\sum I_x^2 & \sum I_x I_y \\
\sum I_x I_y & \sum I_y^2
\end{bmatrix}
\begin{bmatrix}u \\ v\end{bmatrix}
=
-\begin{bmatrix}\sum I_x I_t \\ \sum I_y I_t\end{bmatrix}
$$

Por lo tanto,

$$
u = \frac{-(\sum I_x I_t)(\sum I_y^2) + (\sum I_y I_t)(\sum I_x I_y)}{\text{det}}\quad ; \quad
v = \frac{-(\sum I_y I_t)(\sum I_x^2) + (\sum I_x I_t)(\sum I_x I_y)}{\text{det}}
$$

donde $\text{det} = (\sum I_x^2)(\sum I_y^2) - (\sum I_x I_y)^2 $.


In [ ]:
# Instalación de dependencias
!pip install opencv-python pillow ipywidgets numpy matplotlib --quiet
from IPython.display import display
import ipywidgets as widgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.9 MB/s eta 0:00:00


In [ ]:
# Subir video
# Por ejemplo puede descargar https://github.com/GerardoMunoz/Vision/raw/refs/heads/main/videos/20251008_120434.mp4

from google.colab import files
import cv2, numpy as np, io
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

uploaded = files.upload()
video_path = list(uploaded.keys())[0]
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise Exception('No se pudo abrir el video.')

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Video cargado: {video_path} — {frame_count} frames")

Saving 20251008_120434.mp4 to 20251008_120434.mp4
Video cargado: 20251008_120434.mp4 — 314 frames


In [ ]:
# Selección de frame y ROI interactiva
from ipywidgets import interact, IntSlider

def get_frame(idx):
    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
    ok, f = cap.read()
    if not ok:
        return None
    return cv2.cvtColor(f, cv2.COLOR_BGR2RGB)

current_frame = [None]
roi_coords = [None]

def select_frame(frame_idx=0):
    frame = get_frame(frame_idx)
    if frame is None:
        print("Frame inválido.")
        return
    current_frame[0] = frame
    plt.figure(figsize=(8,6))
    plt.imshow(frame)
    plt.title(f"Frame {frame_idx}")
    plt.axis('off')
    plt.show()
    print("Usa plt.ginput(2) en la siguiente celda para dibujar la ROI (dos puntos opuestos). Guarda las coords en roi_coords.")

interact(select_frame, frame_idx=IntSlider(min=0, max=frame_count-1, step=1, value=0));

interactive(children=(IntSlider(value=0, description='frame_idx', max=313), Output()), _dom_classes=('widget-i…

In [ ]:
# Obtener el shape del video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Shape del video (ancho, alto): ({width}, {height})")

Shape del video (ancho, alto): (1280, 720)


In [ ]:
# 🖱️ Dibujar ROI (click en dos puntos opuestos)
plt.figure(figsize=(8,6))
plt.imshow(current_frame[0])
plt.title("Selecciona dos puntos opuestos del rectángulo ROI (clicks)")
pts = ((0,0),(100,100))#plt.ginput(2)
plt.close()

if len(pts) == 2:
    (x1, y1), (x2, y2) = pts
    x1, x2 = int(min(x1, x2)), int(max(x1, x2))
    y1, y2 = int(min(y1, y2)), int(max(y1, y2))
    roi_coords[0] = (x1, y1, x2, y2)
    print(f"ROI seleccionada: {roi_coords[0]}")
else:
    print("ROI no seleccionada correctamente.")

ROI seleccionada: (0, 0, 100, 100)


In [ ]:
# ⚡ Cálculo del flujo óptico (manual + OpenCV)
from math import atan2, cos, sin, pi

def draw_arrow(draw, x1, y1, x2, y2, color, w=3):
    draw.line((x1,y1,x2,y2), fill=color, width=w)
    angle = atan2(y2-y1, x2-x1)
    head = 10
    draw.polygon([(x2,y2), (x2-head*cos(angle-pi/6), y2-head*sin(angle-pi/6)), (x2-head*cos(angle+pi/6), y2-head*sin(angle+pi/6))], fill=color)

def compute_flow(idx, step=1):
    if roi_coords[0] is None:
        print("Primero selecciona una ROI.")
        return
    x1,y1,x2,y2 = roi_coords[0]
    f1 = get_frame(idx)
    f2 = get_frame(min(idx+step, frame_count-1))
    roi1, roi2 = f1[y1:y2, x1:x2], f2[y1:y2, x1:x2]

    g1 = cv2.cvtColor(roi1, cv2.COLOR_RGB2GRAY).astype(np.float32)
    g2 = cv2.cvtColor(roi2, cv2.COLOR_RGB2GRAY).astype(np.float32)

    h,w = g1.shape
    cx, cy = w//2, h//2
    win = max(7, (min(h,w)//6)*2+1)
    half = win//2

    # Gradientes manuales
    Ix, Iy, It = [], [], []
    for yy in range(cy-half, cy+half):
        for xx in range(cx-half, cx+half):
            xL, xR = max(0,xx-1), min(w-1,xx+1)
            yU, yD = max(0,yy-1), min(h-1,yy+1)
            ix = 0.5*(g1[yy,xR]-g1[yy,xL])
            iy = 0.5*(g1[yD,xx]-g1[yU,xx])
            it = g2[yy,xx]-g1[yy,xx]
            Ix.append(ix); Iy.append(iy); It.append(it)

    Ix, Iy, It = np.array(Ix), np.array(Iy), np.array(It)
    sumIx2 = np.sum(Ix*Ix)
    sumIy2 = np.sum(Iy*Iy)
    sumIxIy = np.sum(Ix*Iy)
    sumIxt = np.sum(Ix*It)
    sumIyt = np.sum(Iy*It)
    det = (sumIx2*sumIy2 - sumIxIy*sumIxIy)
    if abs(det)<1e-6:
        u_manual,v_manual=0,0
    else:
        u_manual = (-sumIxt*sumIy2 - (-sumIyt)*sumIxIy)/det
        v_manual = (sumIx2*(-sumIyt) - sumIxIy*(-sumIxt))/det

    # Farneback
    flow = cv2.calcOpticalFlowFarneback(g1.astype(np.uint8), g2.astype(np.uint8), None, 0.5,3,15,3,5,1.2,0)
    fx, fy = flow[:,:,0], flow[:,:,1]
    u_cv, v_cv = float(np.mean(fx)), float(np.mean(fy))

    # Mostrar
    img = Image.fromarray(f2)
    draw = ImageDraw.Draw(img)
    cx_abs, cy_abs = x1+(x2-x1)/2, y1+(y2-y1)/2
    draw_arrow(draw, cx_abs, cy_abs, cx_abs+u_manual, cy_abs+v_manual, (0,255,0))
    draw_arrow(draw, cx_abs, cy_abs, cx_abs+u_cv, cy_abs+v_cv, (255,0,0))

    plt.figure(figsize=(8,6))
    plt.imshow(img)
    plt.title(f"Flujo óptico — Verde: Manual LK, Rojo: Farneback\nFrame {idx}->{idx+step}")
    plt.axis('off')
    plt.show()
    print(f"Manual LK (u,v)=({u_manual:.3f},{v_manual:.3f}) | Farneback (u,v)=({u_cv:.3f},{v_cv:.3f})")

interact(compute_flow, idx=IntSlider(min=0, max=frame_count-2, step=1, value=0), step=IntSlider(min=1, max=5, value=1));

interactive(children=(IntSlider(value=0, description='idx', max=312), IntSlider(value=1, description='step', m…